In [28]:
import cv2 as cv
import mediapipe as mp
from retinaface import RetinaFace
import numpy as np

# Function to detect faces with RetinaFace using an image path
def detect_faces_with_retina(image_path):
    # Detect faces using RetinaFace from the image path
    resp = RetinaFace.detect_faces(img_path = image_path)
    return resp

# Function to detect and align face using RetinaFace and Mediapipe
def detect_and_align_face(image_path):
    # Load the image
    image = cv.imread(image_path)
    
    if image is None:
        print("Failed to load image")
        return None

    # Detect faces using RetinaFace
    faces = detect_faces_with_retina(image_path)

    if len(faces) == 0:
        print("No faces detected.")
        return None
    
    # Get the first face's bounding box from RetinaFace
    for face_id, face_info in faces.items():
        (x1, y1, x2, y2) = face_info['facial_area']
        face = image[y1:y2, x1:x2]
        
        # Now use Mediapipe for facial landmarks
        with mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5) as face_mesh:
            # Convert to RGB for Mediapipe
            rgb_image = cv.cvtColor(face, cv.COLOR_BGR2RGB)
            face_mesh_results = face_mesh.process(rgb_image)

            if face_mesh_results.multi_face_landmarks:
                landmarks = face_mesh_results.multi_face_landmarks[0].landmark
                
                # Approximate eye positions based on Mediapipe landmarks (indices for left and right eyes)
                left_eye = np.array([int(landmarks[33].x * (x2 - x1)), int(landmarks[33].y * (y2 - y1))])
                right_eye = np.array([int(landmarks[263].x * (x2 - x1)), int(landmarks[263].y * (y2 - y1))])

                print(f"Left eye position: {left_eye}")
                print(f"Right eye position: {right_eye}")

                # Align face based on eye positions
                aligned_face = align_face(face, left_eye, right_eye)
                
                # Resize the aligned face to a fixed size
                resized_face = cv.resize(aligned_face, (128, 128))

                return resized_face
            
    return None

# Function to align the face based on eye positions
def align_face(image, left_eye, right_eye):
    # Calculate the angle between the eyes
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # Calculate the center point between the eyes
    eyes_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)

    # Get the rotation matrix for rotating and aligning the face
    M = cv.getRotationMatrix2D(eyes_center, angle, scale=1)

    # Perform the affine transformation (rotate the image)
    aligned_face = cv.warpAffine(image, M, (image.shape[1], image.shape[0]), flags=cv.INTER_CUBIC)
    
    return aligned_face

# Load and process an example image
image_path = "known_faces/sam.jpg"

processed_face = detect_and_align_face(image_path)

if processed_face is not None:
    # Show the final processed face image
    cv.imshow("Processed Face", processed_face)
    cv.waitKey(0)
    cv.destroyAllWindows()
else:
    print("No face detected")


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
